<a href="https://colab.research.google.com/github/SonOf1998/COVID-19_detection_in_lungs/blob/miki_workspace/COVIDDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import os
from urllib.request import urlretrieve
import requests
import zipfile
import shutil
import random


In [2]:
def rename_all_files_in_folder_with_indices(dir_full_path, files_base_name):
  for idx, file in enumerate(os.listdir(dir_full_path)):
    splitf = file.split('.')
    os.rename(os.path.join(dir_full_path, file), os.path.join(dir_full_path, '{}{:04d}.{}'.format(files_base_name, idx, splitf[-1])))

In [3]:
url = 'https://github.com/muhammedtalo/COVID-19/archive/master.zip'
urlretrieve(url,"master.zip")

local_zip = 'master.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall()
zip_ref.close()

if os.path.exists("master.zip"):
  os.remove("master.zip")

In [4]:
if os.path.exists(os.path.join('COVID-19-master', 'DarkCovidNet  for binary classes.ipynb')):
  os.remove(os.path.join('COVID-19-master', 'DarkCovidNet  for binary classes.ipynb'))
if os.path.exists(os.path.join('COVID-19-master', 'DarkCovidNet model for three classes.ipynb')):
  os.remove(os.path.join('COVID-19-master', 'DarkCovidNet model for three classes.ipynb'))
if os.path.exists(os.path.join('COVID-19-master', 'README.md')):
  os.remove(os.path.join('COVID-19-master', 'README.md'))

In [5]:
destination = os.getcwd()

source = os.path.join('COVID-19-master', 'X-Ray Image DataSet')
shutil.move(source, destination)

os.rmdir('COVID-19-master')

In [6]:
base_dir = 'train_image_dataset'
os.rename('X-Ray Image DataSet', base_dir)

os.rename(os.path.join(base_dir, 'Covid-19'), os.path.join(base_dir, 'covid_19'))
os.rename(os.path.join(base_dir, 'No_findings'), os.path.join(base_dir, 'no_findings'))
os.rename(os.path.join(base_dir, 'Pneumonia'), os.path.join(base_dir, 'pneumonia'))

rename_all_files_in_folder_with_indices(os.path.join(base_dir, 'covid_19'), 'covid_')
rename_all_files_in_folder_with_indices(os.path.join(base_dir, 'no_findings'), 'no_findings_')
rename_all_files_in_folder_with_indices(os.path.join(base_dir, 'pneumonia'), 'pneumonia_')

In [9]:
base_test_dir = 'test_image_dataset'
os.mkdir(base_test_dir)
os.mkdir(os.path.join(base_test_dir, 'covid_19'))
os.mkdir(os.path.join(base_test_dir, 'no_findings'))
os.mkdir(os.path.join(base_test_dir, 'pneumonia'))

In [ ]:
test_split = 0.1

In [29]:
random.seed(1)

test_split = 0.1
current_directory = os.path.join(base_dir, 'covid_19')
new_directory = os.path.join(base_test_dir, 'covid_19')
for i in range(int(len(os.listdir(current_directory)) * test_split)):
  images = os.listdir(current_directory)
  random_image = random.choice(images)
  shutil.move(os.path.join(current_directory, random_image), new_directory)

current_directory = os.path.join(base_dir, 'no_findings')
new_directory = os.path.join(base_test_dir, 'no_findings')
for i in range(int(len(os.listdir(current_directory)) * test_split)):
  images = os.listdir(current_directory)
  random_image = random.choice(images)
  shutil.move(os.path.join(current_directory, random_image), new_directory)

current_directory = os.path.join(base_dir, 'pneumonia')
new_directory = os.path.join(base_test_dir, 'pneumonia')
for i in range(int(len(os.listdir(current_directory)) * test_split)):
  images = os.listdir(current_directory)
  random_image = random.choice(images)
  shutil.move(os.path.join(current_directory, random_image), new_directory)

50